# 伪标签数据划分

In [1]:
import pandas as pd
import random
import os

In [2]:
pse_label_df = pd.read_csv('../复赛融合/submit/sub_8223用于伪标签.csv')

In [3]:
pse_label_df.head()

,id,label
0,0003d01880444716a42159efe73d2c26,1
1,000dbfeed3594d5bb128a21c2078a786,0
2,0010336c4cd44547852edd379487dbda,1
3,001332737eaa443d85b6f59b07d28ed2,1
4,00251d9b40164b638cc784d8e65df52f,1


In [4]:
test_df = pd.read_csv('all_test.csv')
test_df = test_df.loc[:,['id','title','content']]
test_df.head(3)

,id,title,content
0,b9dbbdee402e4dd0a5d1041ff44a001c,“两减六治三提升”专项督查暗访 常州溧阳：矿山生态修复滞后 企业防尘措施不到位,“两减六治三提升”专项督查暗访 常州溧阳：矿山生态修复滞后 企业防尘措施不到位
1,1b48213241064f788dde3c3b359e7622,惊！南宁一家 4 口中毒，母亲口吐白沫！罪魁祸首竟是……,南宁晚报 /zaker 南宁记者 潘国武 文 / 图救援人员赶到邕武路二塘坡时，报警电话却无...
2,f68215c26fed430dab72dd6bea7b52ad,病家求医寄以生死 丹阳人民医院枉为医表,本帖最后由 banished 于 2017-9-28 08:46 编辑 病家求医寄以生死 丹...


In [5]:
def split_train(train_df, train_label_df):
    random.seed(42)
    split_num = 10
    train_df=train_df.merge(train_label_df,on='id',how='left')
    train_df['label']=train_df['label'].fillna(-1)
    train_df=train_df[train_df['label']!=-1]
    train_df['label']=train_df['label'].astype(int)
    train_df['content']=train_df['content'].fillna('无')
    train_df['title']=train_df['title'].fillna('无')
    
    index=set(range(train_df.shape[0]))
    K_fold=[]
    for i in range(split_num):
        if i == split_num-1:
            tmp=index
        else:
            tmp=random.sample(index,int(1.0/split_num*train_df.shape[0]))
        index=index-set(tmp)
        print("Number:",len(tmp))
        K_fold.append(tmp)


    for i in range(split_num):
        print("Fold",i)
        data_dir = "data_pse_10fold/data_pse_42_{}".format(i)
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        dev_index=list(K_fold[i])
        train_df.iloc[dev_index].to_csv((data_dir+"/train.csv").format(i))


In [6]:
split_train(test_df, pse_label_df)

Number: 735
Number: 735
Number: 735
Number: 735
Number: 735
Number: 735
Number: 735
Number: 735
Number: 735
Number: 741
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9


# 伪标签和原训练集拼接

In [11]:
import pandas as pd
import os
def pse_merge_train(train_id, pse_id):
#     train_id, pse_id = 3,3
    train_dir = "data_StratifiedKFold_42/data_origin_{}/".format(train_id)
    pse_dir = "data_pse_10fold/data_pse_42_{}/".format(pse_id)
    train_pse_dir = "origin_data_pse/origin_data_train{}_pse{}/".format(train_id,pse_id)
    os.system("cp -r "+train_dir+" "+train_pse_dir)

    train_df = pd.read_csv(train_dir+'train.csv')
    pse_df = pd.read_csv(pse_dir+'train.csv')

    rst = pd.concat([train_df, pse_df],ignore_index=True)
    rst.loc[:,['id','title','content','label']].to_csv(train_pse_dir + "train.csv")

In [12]:
rst = pse_merge_train(3, 3)
rst = pse_merge_train(0, 0)
rst = pse_merge_train(1, 1)
rst = pse_merge_train(2, 2)
rst = pse_merge_train(4, 4)